In [ ]:
# ! pip install -r requirements.txt 


Defaulting to user installation because normal site-packages is not writeable
  Using cached click-7.1.2-py2.py3-none-any.whl (82 kB)
     |████████████████████████████████| 291 kB 2.4 MB/s eta 0:00:01
  Created wheel for pyppdf: filename=pyppdf-0.1.2-py3-none-any.whl size=14088 sha256=98c1eb0b0fe8984419daebd28e2ffec605fc205a41ad1a5ca85cf0ca7d4ad8e2
  Stored in directory: /home/tuhp/.cache/pip/wheels/eb/00/28/373d9d5e10dd2f5ef1a7fba3735b26f9f7f57be23e72c3f570
  Created wheel for litereval: filename=litereval-0.0.11-py3-none-any.whl size=7057 sha256=2bfd631a7f4f0f8ee9dfc0ec52749debe4ce22befeffc6b12f8e2dbd1d6356f5
  Stored in directory: /home/tuhp/.cache/pip/wheels/69/6b/e8/10ae8c15ecac33379af3198cd008df6f1196d384e209cae5a6
Successfully built pyppdf litereval
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [7]:
# import libraries
from requests_html import HTMLSession
from requests_html import AsyncHTMLSession
import pyppdf.patch_pyppeteer
import requests
import json
import re
from tqdm.notebook import tqdm
import csv


# declare global varibles 
SESSION = HTMLSession()

asession = AsyncHTMLSession()
HOME = 'http://www.phimmoizz.net/'

#### Prototype Functions:
* `get_movies_on_page(links,session = SESSION)` : return list of movie urls from link:
* `get_data_from_movie(url,session=SESSION)`:  return information of movie in url; return type: dictionary 
* `links_crawler(output='links.txt')`: crawl all movie links on http://www.phimmoizz.net/phim-le/ (while crawling, append links to file output)
* `movie_data_crawler(input='links.txt')`: crawl data of movies from file input contain links of movies

#### Define functions

In [2]:
def get_movies(link,session = SESSION):
    r = session.get(link)
    return [HOME + e.attrs['href'] for e in r.html.find('.list-movie')[0].find('a') ]

# exammple 
get_movies('http://www.phimmoizz.net/phim-le/page-1.html')

['http://www.phimmoizz.net/phim/thong-long-ma-2-12027/',
 'http://www.phimmoizz.net/phim/phim-doraemon-nobita-va-nhung-ban-khung-long-moi-9271/',
 'http://www.phimmoizz.net/phim/intersection-12028/',
 'http://www.phimmoizz.net/phim/cung-lam-anh-hung-12023/',
 'http://www.phimmoizz.net/phim/soul-9807/',
 'http://www.phimmoizz.net/phim/them-mot-chau-nua-nhe-12022/',
 'http://www.phimmoizz.net/phim/ky-nghi-le-vang-ky-nghi-nho-doi-12021/',
 'http://www.phimmoizz.net/phim/ak-vs-ak-12024/',
 'http://www.phimmoizz.net/phim/nu-than-chien-binh-1984-9573/',
 'http://www.phimmoizz.net/phim/doi-quan-mot-nguoi-12015/',
 'http://www.phimmoizz.net/phim/hon-ca-mot-gia-dinh-me-bau-sieu-ngau-12012/',
 'http://www.phimmoizz.net/phim/ta-ma-va-bong-toi-12005/',
 'http://www.phimmoizz.net/phim/lap-lanh-troi-dem-12008/',
 'http://www.phimmoizz.net/phim/bien-nien-su-giang-sinh-phan-hai-11879/',
 'http://www.phimmoizz.net/phim/bien-nien-su-giang-sinh-8041/',
 'http://www.phimmoizz.net/phim/cai-ten-khac-sau-tro

In [2]:
import pandas as pd
pd.read_json('data.json',lines=True).to_csv('data.csv',index=False)

In [38]:
def get_data_from_movie(url,session=SESSION):
    '''
        return dictionary attributes:
                name : str
                ,id: str
                ,IMDb : float
                ,IMDb_votes : int
                ,AW: float
                ,AW_votes: int
                ,duration: int (minutes)
                ,year : int
                ,relase : datetime
                ,nations : str
                ,quality : str
                ,resolution: str
                ,language: str
                ,genres: str
                ,company: str
                ,star: int
                ,star_votes: int
                ,recommendations : list ID (class of model)
    '''

    mapping_name = {
        'Thể loại': 'genres',
        'Ngôn ngữ': 'language',
        'Độ phân giải':'resolution',
#         '': 'star',
#         '': 'star_votes',
        'Thời lượng': 'duration',
        'Chất lượng': 'quality',
        'Quốc gia': 'nations',
        'Công ty SX': 'company',
        'Ngày phát hành': 'relase',
        'Đạo diễn':'director',
#         '': 'recommendations',
#         '': 'AW_votes',
#         '': 'AW',
        'Điểm IMDb': 'IMDb',
        'Số người đánh giá': 'IMDb_votes',
    }
    r =  session.get(url)    
    result = {}
    
    for attr, info in zip(r.html.find('.movie-dt'), r.html.find('.movie-dd')):
        if attr.text[:-1] not in mapping_name.keys():
            continue
        result[mapping_name[attr.text[:-1]]] = info.text.strip(' ()')
    result['vietnamese_name'] = r.html.find('.title-1')[0].text
    result['name'] = r.html.find('.title-2')[0].text
    result['year'] = r.html.find('.title-year')[0].text.strip(' ()')
    
    result['id'] = r.url.strip('/').split('-')[-1]
    
    result['star'] = r.html.find('#score_current')[0].attrs['value']
    #result['star_votes'] = r.html.find('.rating-text')[0].text.split()[-2]
    
    
   
    
    if 'IMDb_votes' in result.keys():
        result['IMDb_votes'] = re.sub('[^\d]+', '', str(result['IMDb_votes']))
    else:
        result['IMDb_votes'] = None
    result['duration'] = re.sub('[^\d]+', '', str(result['duration']))
    
    result['film_content'] = r.html.find('#film-content')[0].text
    result['actors'] = ", ".join([ actor.text for actor in r.html.find('.actor-name-a')])
#     print(result)
    return result

# example 
url = 'http://www.phimmoizz.net/phim/diep-van-bac-thay-vo-thuat-11979/'
get_data_from_movie(url)


{'IMDb': '4.3',
 'IMDb_votes': '166',
 'director': 'Liming Li,',
 'nations': 'Trung Quốc,',
 'relase': '23/12/2019',
 'duration': '84',
 'quality': 'Bản đẹp',
 'resolution': 'HD 720p',
 'language': 'Thuyết minh',
 'genres': 'Phim hành động, Phim võ thuật, Phim hồi hộp-Gây cấn, Phim thuyết minh, Phim lẻ',
 'company': 'Chưa rõ',
 'vietnamese_name': 'Diệp Vấn: Bậc Thầy Võ Thuật',
 'name': 'Ip Man: Kung Fu Master',
 'year': '2019',
 'id': '11979',
 'star': '5.96',
 'film_content': 'IP MAN: KUNG FU MASTER quay trở lại những ngày đầu của Ip trước Cách mạng Cộng sản năm 1949. Ip, do Dennis To thể hiện lần thứ ba với vai võ sĩ nổi tiếng kèm cặp Lý Tiểu Long, lúc đó là đội trưởng cảnh sát bị buộc tội giết người của một tên cướp tàn nhẫn nhưng danh dự, và được nhắm mục tiêu báo thù bởi đứa con gái nguy hiểm của mình. Bị buộc phải từ bỏ lực lượng, Ip cũng sớm phải đối mặt với sự xuất hiện của quân đội Nhật Bản tại Quảng Châu.Trong thời gian làm đội trưởng cảnh sát ở Phật Sơn, Ip Man là mục tiêu c

In [39]:
def links_crawler(output='links.txt', session=SESSION):
    finish = False
    page = 1 
    while  not finish :
        link = HOME+ '/phim-le/page-{}'.format(page) + '.html'
        r = session.get(link)
        list_movie = r.html.find('.list-movie',first=True).find('a')
        if len(list_movie) > 0:  
            # periodly print progess 
            if page % 50 == 0 :
                print('Current page: {}'.format(page))
            
            # write to file
            with open(output,'a') as f:
                f.writelines([ HOME + e.attrs['href'] + '\n' for e in list_movie])
            page +=1
            
        else:
            print('Total page: {}'.format(page))
            finish = True

In [125]:
links_crawler()

Current page: 50
Current page: 100
Current page: 150
Total page: 200


In [123]:
def movie_data_crawler(input='links.txt', session=SESSION):
    film_links = []
    with open(input, 'r') as f:
        film_links = f.readlines()
    film_links = [link[:-1] for link in film_links] # drop \n
    flag = False
    film_data = []
    par = tqdm(enumerate(film_links),total =len(film_links))
    csv_columns = list(get_data_from_movie(film_links[0]).keys())
    with open('data.csv', 'a+') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=csv_columns)
        writer.writeheader()
    
    for index,link in par: 
        try :
            film_dict = get_data_from_movie(link)            
            pd.DataFrame(film_dict,columns =csv_columns,index=[index]).to_csv('data.csv',mode='a+',header=False,index=False)
        except:
            flag = True
            with open('errors.txt','a+') as f:
                f.write('{}\n'.format(index))
    print("Try scrawl data from links that get error")
    if flag :
        with open('errors.txt','r') as f:
            error = f.readlines()
            index = [int(e[:-1]) for e in error]
            for i in index: 
                pd.DataFrame(film_dict,columns =csv_columns,index=[i]).to_csv('data.csv',mode='a+',header=False,index=False)
    return 0
    
        
#     print(film_links)

In [126]:
movie_data_crawler()


Try scrawl data from links that get error


0

# Read data

In [127]:
import pandas as pd
import numpy as np

df = pd.read_csv('data.csv')
df.head()

ParserError: Error tokenizing data. C error: Expected 17 fields in line 216, saw 18


In [ ]:
len(df)

In [ ]:
df.isna().sum()

In [ ]:
df.describe()

In [ ]:
df.dtypes

In [ ]:
df.director.nunique()

In [ ]:
genres = df.genres.apply(lambda x:x.split(', ')).explode().unique().tolist()

In [ ]:
import numpy as np
labels = np.zeros((len(df),len(genres)))


In [ ]:
lb = []
for i,g in df.genres.apply(lambda x:x.split(', ')).explode().apply(lambda x: genres.index(x)).items():
  labels[i][g] = 1
labels.shape

In [ ]:
df = pd.concat([df,pd.DataFrame(labels,columns =genres )], axis = 1)

In [ ]:
df.drop(['genres'],inplace = True,axis = 1)
df.shape

In [ ]:
df.columns

0                   The Teacher (My Soul Is Made Of Love)
1                                             Last Letter
6                                                Ammonite
10                  The Teacher (My Soul Is Made Of Love)
11                                            Last Letter
                              ...                        
5586                                              Titanic
5619                          You Are the Apple of My Eye
5686                                       The Best Offer
5877                                       A Werewolf Boy
5930    The Concubine / Royal Concubine: Concubine of ...
Name: name, Length: 699, dtype: object